# Delete SageMaker Resources

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import base64
import os
from typing import Dict, List

import boto3
from dotenv import find_dotenv, load_dotenv

In [ ]:
%aimport src.s3.buckets
import src.s3.buckets as s3b

%aimport src.iam.iam_roles
import src.iam.iam_roles as iamr

%aimport src.ec2.ec2_instances_sec_groups
import src.ec2.ec2_instances_sec_groups as ec2h

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
aws_region = os.getenv("AWS_REGION")

## About

In this notebook, the following resources related to AWS SageMaker will be destroyed
- SageMaker notebook instance
- SageMaker notebook lifecycle configuration
- CloudWatch Logging group for the SageMaker instance
- EC2 Security group for the SageMaker instance
- S3 bucket

### Pre-Requisites
1. As mentioned in `README.md`, the following environment variables should be set with the user's AWS credendials ([1](https://docs.aws.amazon.com/sdk-for-php/v3/developer-guide/guide_credentials_environment.html), [2](https://docs.aws.amazon.com/sdk-for-php/v3/developer-guide/guide_credentials_profiles.html))
   - `AWS_ACCESS_KEY_ID`
   - `AWS_SECRET_KEY`
   - `AWS_REGION`

   These credentials must be associated to a user group whose users have been granted programmatic access to AWS resources. In order to configure this for an IAM user group, see the documentation [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html#id_users_create_console).

### Notes
1. All resources must exist in the same AWS region (specified by the environment variable `AWS_REGION`).

## User Inputs

In [ ]:
# S3
s3_bucket_name = "sagemakerdemoedesz4321"

# IAM Role
iam_role_name = "AmazonSageMaker-ExecutionRole-20211222T235985"

# EC2 Security Groups
sg_group_name = "mysgname"

# Sagemaker Lifecycle
nb_lifecycle_name = "mynbconfig"
nb_instance_name = "mydemo"
nb_instance_tags = [{"Key": "Name", "Value": nb_instance_name}]

# Cloud Watch
cw_log_group_name = "/aws/sagemaker/NotebookInstances"

## SageMaker

### Stop Notebook Instance

In [ ]:
def stop_sagemaker_nb_instance(nb_instance_name: str, aws_region: str) -> None:
    client = boto3.client("sagemaker", region_name=aws_region)
    stop_nb_response = client.stop_notebook_instance(NotebookInstanceName=nb_instance_name)
    return stop_nb_response

In [ ]:
%%time
stop_nb_response = stop_sagemaker_nb_instance(nb_instance_name, aws_region)
stop_nb_response

### Delete Notebook Instance

In [ ]:
def delete_sagemaker_nb_instance(nb_instance_name: str, aws_region: str):
    client = boto3.client("sagemaker", region_name=aws_region)
    response_nb_list = client.list_notebook_instances(NameContains=nb_instance_name)
    nb_dict = response_nb_list["NotebookInstances"][0]
    nb_state = nb_dict["NotebookInstanceStatus"]
    if nb_state not in ["Stopped"]:
        print(f"Sagemaker Instance in state {nb_state}. Could not be deleted. Did nothing.")
    else:
        nb_delete_response = client.delete_notebook_instance(
            NotebookInstanceName=nb_instance_name
        )
        print("Sagemaker Instance Deleted.")

In [ ]:
%%time
delete_sagemaker_nb_instance(nb_instance_name, aws_region)

### Delete Lifecycle Config

In [ ]:
def delete_sagemaker_nb_instance_lifecycle_config(nb_lifecycle_name: str, aws_region: str) -> None:
    client = boto3.client("sagemaker", region_name=aws_region)
    nb_lifecycle_list = client.list_notebook_instance_lifecycle_configs(
        NameContains=nb_lifecycle_name
    )
    if nb_lifecycle_list["NotebookInstanceLifecycleConfigs"]:
        nb_lifecycle_config_name = nb_lifecycle_list["NotebookInstanceLifecycleConfigName"]
        nb_config_delete_response = client.delete_notebook_instance_lifecycle_config(
            NotebookInstanceLifecycleConfigName=nb_lifecycle_name
        )
        print(f"Deleted lifecycle config: {nb_lifecycle_config_name}")
    else:
        print("Did not find any lifecycle configurations. Did nothing.")

In [ ]:
%%time
delete_sagemaker_nb_instance_lifecycle_config(nb_lifecycle_name, aws_region)

### Delete the CloudWatch Logging Group for SageMaker Instance

In [ ]:
def delete_cw_log_group_stream(cw_log_group_name: str, aws_region: str) -> Dict:
    """Delete CloudWatch Logging Group."""
    cw_logs_client = boto3.client("logs", region_name=aws_region)
    cw_log_deletion_response = cw_logs_client.delete_log_group(
        logGroupName=cw_log_group_name
    )

In [ ]:
%%time
delete_cw_log_group_stream(cw_log_group_name, aws_region)

### Delete EC2 Security Group for the SageMaker Instance

In [ ]:
def delete_sagemaker_nb_instance_security_group(
    nb_instance_name: str, aws_region: str, sg_group_id: str
) -> None:
    client = boto3.client("sagemaker", region_name=aws_region)
    response_nb_list = client.list_notebook_instances(NameContains=nb_instance_name)
    if not response_nb_list["NotebookInstanceLifecycleConfigs"]:
        print(
            "Found Notebook instance that depends on "
            f"Security Group: {sg_group_id}. Did nothing."
        )
    else:
        ec2h.delete_sg(sg_group_id, aws_region)

In [ ]:
%%time
sg_filter = dict(Filters=[{"Name": "tag:Name", "Values": [sg_group_name]}])
sg_id_list = get_security_group_ids(aws_region, sg_filter)
delete_sagemaker_nb_instance_security_group(nb_instance_name, aws_region, sg_id_list[0])

## Delete the S3 Bucket

In [ ]:
%%time
s3b.delete_s3_bucket(s3_bucket_name, aws_region)